In [36]:
import pandas as pd
import requests

url = "https://raw.githubusercontent.com/Papagoat/brain-assessment/main/restaurant_data.json"
response = requests.get(url)
restaurant_data = response.json()
country_path = 'Country-Code.xlsx'
country_df =   pd.read_excel(country_path)


In [37]:
restaurants_list = []

for entry in restaurant_data:
    for restaurant in entry['restaurants']:
        info = {
            'Restaurant Id': restaurant['restaurant']['R']['res_id'],
            'Restaurant Name': restaurant['restaurant']['name'],
            'Country Code': restaurant['restaurant']['location']['country_id'], 
            'City': restaurant['restaurant']['location']['city'],
            'User Rating Votes': restaurant['restaurant']['user_rating']['votes'],
            'User Aggregate Rating': float(restaurant['restaurant']['user_rating']['aggregate_rating']),
            'Cuisines': restaurant['restaurant']['cuisines']
        }
        restaurants_list.append(info)
restaurants_df = pd.DataFrame(restaurants_list)
combined_df = pd.merge(restaurants_df, country_df, on='Country Code', how='left')
final_df = combined_df.drop('Country Code', axis=1)
final_df.to_csv('restaurants.csv', index=False)


In [38]:
from datetime import datetime

event_list = []

start_date = datetime(2019, 4, 1)
end_date = datetime(2019, 4, 30)

for entry in restaurant_data:
    for restaurant in entry['restaurants']:
        if 'zomato_events' in restaurant['restaurant']:
            for event in restaurant['restaurant']['zomato_events']:
                event_start = datetime.strptime(event['event']['start_date'], '%Y-%m-%d')
                event_end = datetime.strptime(event['event']['end_date'], '%Y-%m-%d')
                if start_date <= event_start <= end_date or start_date <= event_end <= end_date:
                    if 'photos' in event['event'] and len(event['event']['photos']) > 0:
                        photo_url = event['event']['photos'][0]['photo']['url']
                    else:
                        photo_url = "NA"
                    info = {
                        'Event Id': event['event']['event_id'],
                        'Restaurant Id': restaurant['restaurant']['R']['res_id'],
                        'Restaurant Name': restaurant['restaurant']['name'],
                        'Photo URL': photo_url,
                        'Event Title': event['event']['title'],
                        'Event Start Date': event['event']['start_date'],
                        'Event End Date': event['event']['end_date']
                    }
                    event_list.append(info)

event_df = pd.DataFrame(event_list)

event_df.fillna("NA", inplace=True)

event_df.to_csv('restaurant_events.csv', index=False)